<a href="https://colab.research.google.com/github/11kartheek/gpt2dummy/blob/main/infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!git clone https://huggingface.co/spaces/Kartheekb7/s21-gpt3

Cloning into 's21-gpt3'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 19 (delta 4), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (19/19), 434.03 KiB | 4.13 MiB/s, done.
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'


Exiting because of "interrupt" signal.
^C


In [14]:
%cd /content/s21-gpt3

/content/s21-gpt3


In [3]:
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.8 MB/s eta 0:00:00


In [15]:
import torch
import tiktoken
from model import *

enc = tiktoken.get_encoding('gpt2')
model = torch.load('model.pt',map_location='cpu')


def response(message = "Hello, I'm a language model", num_return_sequences = 5,max_length = 30,top_k = 50):
  tokens = enc.encode(message)
  tokens = torch.tensor(tokens, dtype= torch.long) # (8,) #check tiktoken app
  tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) # (5, 8)
  x = tokens.to('cpu')
  torch.manual_seed(42)
  torch.cuda.manual_seed(42)
  while x.size(1) < max_length:
      # forward the model to get the logits
      with torch.no_grad():
          logits = model(x)[0] # (B, T, vocab_size)
          # take the logits at the last position
          logits = logits[:, -1, :] # (B, vocab_size)
          # get the probabilities
          probs = F.softmax(logits, dim=-1)
          # do top-k sampling of 50 (huggingface pipeline default)
          # topk_probs here becomes (5, 50), topk_indices is (5, 50)
          topk_probs, topk_indices = torch.topk(probs, top_k, dim=-1)
          # select a token from the top-k probabilities
          # note: multinomial does not demand the input to sum to 1
          ix = torch.multinomial(topk_probs, 1) # (B, 1)
          # gather the corresponding indices
          xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
          # append to the sequence
          x = torch.cat((x, xcol), dim=1)

  # print the generated text
  return_text = ""
  for i in range(num_return_sequences):
      tokens = x[i, :max_length].tolist()
      decoded = enc.decode(tokens)
      return_text = return_text + ">"+ decoded +"\n"
  return return_text

In [16]:
response(message = "Hello, I'm a language model", num_return_sequences = 5,max_length = 30,top_k = 50)

">Hello, I'm a language models all\nThen never take truthelit since law for it rather hearal\nShAUA: go't\n>Hello, I'm a language models all\nWhat but say our woman was power that have kill his\nWhere thinks well like oneathe my power\n>Hello, I'm a language model or drunk:\nThat possess it might be taneily art but we play\nAnd never lound art a\n>Hello, I'm a language models all\nUnless heark as I:\nYet ere he thinks my part of traark know she did\n>Hello, I'm a language models life\nWhat news'T' speak an that do be brought it know he hath in we burn once I\n"

In [11]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.3 MB/s 

In [18]:
import gradio as gr
import torch
import tiktoken
enc = tiktoken.get_encoding('gpt2')
model = torch.load('model.pt',map_location='cpu')

# Function to generate text
def generate_text(prompt, top_k, max_return_sequences, max_tokens):
    ans = response(message = prompt, num_return_sequences = max_return_sequences,max_length = max_tokens,top_k = top_k)


    return ans

# Create Gradio interface
iface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(lines=5, label="Input Text"),
        gr.Slider(minimum=1, maximum=100, value=50, step=1, label="Top-k"),
        gr.Slider(minimum=1, maximum=5, value=1, step=1, label="Return Sequences"),
        gr.Slider(minimum=10, maximum=150, value=20, step=5, label="Max Tokens")
    ],
    outputs=gr.Textbox(label="Generated Text"),
    title="GPT-2 Text Generator",
    description="Generate text using GPT-2 model with adjustable parameters.",
    examples=[["Hello, I'm a language model"]],
)

# Launch the interface
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3bbd5dd7196436d398.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Load the GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Function to generate text
def generate_text(prompt, top_p, top_k, max_return_sequences, max_tokens):
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate text
    output_sequences = model.generate(
        input_ids=inputs["input_ids"],
        do_sample=True,
        top_p=top_p,
        top_k=top_k,
        max_length=max_tokens,
        num_return_sequences=max_return_sequences
    )

    # Decode generated sequences
    generated_texts = [tokenizer.decode(seq, skip_special_tokens=True) for seq in output_sequences]

    return generated_texts

# Create Gradio interface
iface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(lines=5, label="Input Text"),
        gr.Slider(minimum=0.0, maximum=1.0, default=0.9, step=0.01, label="Top-p (nucleus sampling)"),
        gr.Slider(minimum=1, maximum=100, default=50, step=1, label="Top-k"),
        gr.Slider(minimum=1, maximum=10, default=1, step=1, label="Max Return Sequences"),
        gr.Slider(minimum=10, maximum=1000, default=200, step=10, label="Max Tokens")
    ],
    outputs=gr.Textbox(label="Generated Text"),
    title="GPT-2 Text Generator",
    description="Generate text using GPT-2 model with adjustable parameters.",
)

# Launch the interface
iface.launch()
